In [ ]:
import numpy as np
import torch
import time
import matplotlib.pyplot as plt
from tabulate import tabulate

# Configuración del estilo de las gráficas
plt.style.use('ggplot')
plt.rcParams.update({'font.size': 12})

def ejecutar_pruebas():
    """Ejecuta todas las pruebas de rendimiento y muestra los resultados"""
    
    print("=" * 80)
    print("COMPARATIVA DE RENDIMIENTO: GPU vs CPU PARA CÁLCULOS MATRICIALES")
    print("=" * 80)
    
    # Verificar si hay GPU disponible
    cuda_disponible = torch.cuda.is_available()
    if cuda_disponible:
        dispositivo_gpu = torch.cuda.get_device_name(0)
        print(f"\n✅ GPU detectada: {dispositivo_gpu}")
        print(f"   Versión CUDA: {torch.version.cuda}")
    else:
        print("\n❌ No se detectó ninguna GPU. Solo se ejecutarán pruebas en CPU.")
        print("   Para usar GPU, asegúrate de tener instalado PyTorch con soporte CUDA.")
    
    print("\n" + "-" * 80)
    print("INFORMACIÓN DEL SISTEMA")
    print("-" * 80)
    
    # Obtener información del sistema
    import platform
    import psutil
    import cpuinfo
    
    info_cpu = cpuinfo.get_cpu_info()
    
    info_sistema = [
        ["Sistema Operativo", platform.platform()],
        ["CPU", info_cpu.get('brand_raw', platform.processor())],
        ["Núcleos Físicos", psutil.cpu_count(logical=False)],
        ["Núcleos Totales", psutil.cpu_count()],
        ["RAM Total", f"{psutil.virtual_memory().total / (1024**3):.2f} GB"],
    ]
    
    if cuda_disponible:
        info_sistema.append(["GPU", dispositivo_gpu])
        info_sistema.append(["Memoria GPU", f"{torch.cuda.get_device_properties(0).total_memory / (1024**3):.2f} GB"])
        info_sistema.append(["Versión CUDA", torch.version.cuda])
    
    # Mostrar información del sistema
    print(tabulate(info_sistema, tablefmt="simple"))
    
    # Definir tamaños de matrices a probar
    tamaños = [500, 1000, 2000, 4000, 6000, 8000]
    resultados = []
    
    print("\n" + "-" * 80)
    print("PRUEBAS DE RENDIMIENTO: MULTIPLICACIÓN DE MATRICES")
    print("-" * 80)
    
    # Realizar pruebas para cada tamaño
    for tamaño in tamaños:
        print(f"\nPrueba con matrices de {tamaño}x{tamaño}:")
        
        # Crear matrices de prueba
        A_np = np.random.rand(tamaño, tamaño).astype(np.float32)
        B_np = np.random.rand(tamaño, tamaño).astype(np.float32)
        
        # ----- Prueba en CPU (NumPy) -----
        print(f"  🔄 Ejecutando en CPU...", end="", flush=True)
        
        # Calentamiento para CPU
        _ = np.matmul(A_np, B_np)
        
        # Medir tiempo en CPU
        inicio_cpu = time.time()
        C_np = np.matmul(A_np, B_np)
        tiempo_cpu = time.time() - inicio_cpu
        
        print(f" completado en {tiempo_cpu:.4f} segundos")
        
        # ----- Prueba en GPU (PyTorch) -----
        tiempo_gpu = float('nan')  # Valor predeterminado si no hay GPU
        aceleracion = float('nan')
        
        if cuda_disponible:
            print(f"  🔄 Ejecutando en GPU...", end="", flush=True)
            
            # Convertir a tensores de PyTorch y mover a GPU
            A_torch = torch.from_numpy(A_np).to('cuda')
            B_torch = torch.from_numpy(B_np).to('cuda')
            
            # Calentamiento para GPU (primera ejecución suele ser más lenta)
            _ = torch.matmul(A_torch, B_torch)
            torch.cuda.synchronize()
            
            # Medir tiempo en GPU
            inicio_gpu = time.time()
            C_torch = torch.matmul(A_torch, B_torch)
            torch.cuda.synchronize()  # Esperar a que la operación en GPU termine
            tiempo_gpu = time.time() - inicio_gpu
            
            # Calcular aceleración
            aceleracion = tiempo_cpu / tiempo_gpu
            
            print(f" completado en {tiempo_gpu:.4f} segundos")
            print(f"  ⚡ Aceleración GPU vs CPU: {aceleracion:.2f}x")
            
            # Verificar que los resultados sean similares (opcional)
            C_torch_cpu = C_torch.cpu().numpy()
            error_relativo = np.mean(np.abs(C_np - C_torch_cpu) / (np.abs(C_np) + 1e-10))
            print(f"  ✓ Error relativo: {error_relativo:.2e} (debe ser cercano a cero)")
        
        # Guardar resultados
        resultados.append({
            'tamaño': tamaño,
            'tiempo_cpu': tiempo_cpu,
            'tiempo_gpu': tiempo_gpu if cuda_disponible else None,
            'aceleracion': aceleracion if cuda_disponible else None
        })
    
    # ----- Mostrar resultados en forma de tabla -----
    print("\n" + "-" * 80)
    print("RESUMEN DE RESULTADOS")
    print("-" * 80)
    
    tabla_datos = []
    for r in resultados:
        if cuda_disponible:
            tabla_datos.append([
                r['tamaño'], 
                f"{r['tiempo_cpu']:.4f}s", 
                f"{r['tiempo_gpu']:.4f}s", 
                f"{r['aceleracion']:.2f}x"
            ])
        else:
            tabla_datos.append([
                r['tamaño'], 
                f"{r['tiempo_cpu']:.4f}s", 
                "N/A", 
                "N/A"
            ])
    
    headers = ["Tamaño Matriz", "Tiempo CPU", "Tiempo GPU", "Aceleración"]
    print(tabulate(tabla_datos, headers=headers, tablefmt="simple"))
    
    # ----- Generar gráficas de rendimiento -----
    if cuda_disponible:
        generar_graficas(resultados)
    
    print("\n" + "=" * 80)
    print("ANÁLISIS COMPLETADO")
    print("=" * 80)
    
    # Interpretación de resultados
    print("\nInterpretación de los resultados:")
    
    if cuda_disponible:
        aceleracion_promedio = sum(r['aceleracion'] for r in resultados) / len(resultados)
        max_aceleracion = max(r['aceleracion'] for r in resultados)
        tamaño_max_aceleracion = resultados[[r['aceleracion'] for r in resultados].index(max_aceleracion)]['tamaño']
        
        print(f"\n1. La GPU proporciona una aceleración promedio de {aceleracion_promedio:.2f}x respecto a la CPU.")
        print(f"2. La máxima aceleración ({max_aceleracion:.2f}x) se observó con matrices de {tamaño_max_aceleracion}x{tamaño_max_aceleracion}.")
        print("3. La ventaja de la GPU es más notable con matrices grandes, donde el paralelismo es más efectivo.")
        print("4. Para matrices pequeñas, la sobrecarga de transferir datos a la GPU puede reducir el beneficio.")
    else:
        print("\n1. No se detectó GPU para realizar comparaciones.")
        print("2. Los tiempos de CPU aumentan de forma cuadrática o cúbica con el tamaño de la matriz.")
        print("3. Para cálculos intensivos con matrices grandes, una GPU podría ofrecer mejoras significativas.")

def generar_graficas(resultados):
    """Genera gráficas comparativas de rendimiento"""
    
    tamaños = [r['tamaño'] for r in resultados]
    tiempos_cpu = [r['tiempo_cpu'] for r in resultados]
    tiempos_gpu = [r['tiempo_gpu'] for r in resultados]
    aceleraciones = [r['aceleracion'] for r in resultados]
    
    # Figura principal con dos subgráficas
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Gráfica 1: Tiempos de ejecución
    ax1.plot(tamaños, tiempos_cpu, 'o-', color='#E24A33', linewidth=2, label='CPU (NumPy)')
    ax1.plot(tamaños, tiempos_gpu, 'o-', color='#348ABD', linewidth=2, label='GPU (PyTorch)')
    ax1.set_xlabel('Tamaño de Matriz')
    ax1.set_ylabel('Tiempo (segundos)')
    ax1.set_title('Tiempos de Ejecución: CPU vs GPU')
    ax1.grid(True, alpha=0.3)
    ax1.legend()
    
    # Gráfica 2: Aceleración
    ax2.bar(tamaños, aceleraciones, color='#7A68A6', alpha=0.7)
    ax2.axhline(y=1, color='r', linestyle='--', alpha=0.5)
    ax2.set_xlabel('Tamaño de Matriz')
    ax2.set_ylabel('Aceleración (veces más rápido)')
    ax2.set_title('Aceleración GPU vs CPU')
    for i, v in enumerate(aceleraciones):
        ax2.text(tamaños[i], v + 0.2, f"{v:.1f}x", ha='center', fontsize=10)
    
    plt.tight_layout()
    plt.savefig('comparativa_gpu_cpu.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("\n✅ Gráfica guardada como 'comparativa_gpu_cpu.png'")

if __name__ == "__main__":
    ejecutar_pruebas()
